In [9]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install requests
!pip install tqdm
from tqdm import tqdm
import os
import sys
from pathlib import Path
import requests
import threading

def download(url, filename=None):
    """
    Downloads a file from a given URL with a progress bar.

    Args:
        url (str): The URL of the file to download.
        filename (str, optional): The name to save the file as.
                                  If None, the original filename from the URL is used.
    """
    try:
        # Use allow_redirects=True to follow redirects, as some URLs might redirect to the actual file.
        response = requests.head(url, allow_redirects=True)
        filesize = int(response.headers["Content-Length"])

        # Determine the filename to use
        if filename is None:
            filename = os.path.basename(response.url) # Use the final redirected URL to get the filename

        print(f"Downloading {filename} ({filesize} bytes)...")

        # Define download path
        home_path = "/content/drive/MyDrive"
        sub_path = "downloads"
        os.makedirs(os.path.join(home_path, sub_path), exist_ok=True)
        dl_path = os.path.join(home_path, sub_path, filename)

        chunk_size = 1024
        with requests.get(url, stream=True, allow_redirects=True) as r, open(dl_path, "wb") as f, tqdm(
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
            total=filesize,
            file=sys.stdout,
            desc=filename
        ) as progress:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk: # Filter out keep-alive new chunks
                    datasize = f.write(chunk)
                    progress.update(datasize)
    except requests.exceptions.RequestException as e:
        print(f"\nError downloading {url}: {e}")
    except KeyError:
        print(f"\nError: Could not determine file size for {url}. The 'Content-Length' header was not found.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

In [20]:
urls_and_filenames = []
while True:
    url = input("Enter file URL (or type 'exit' to finish): ")
    if url.lower() == "exit":
        break

    modify_filename = input("Do you want to modify the file name? (y/n): ").lower()
    if modify_filename == 'y':
        new_filename = input("Enter new file name: ")
        urls_and_filenames.append((url, new_filename))
        print(f"URL and new filename added: {url} -> {new_filename}")
    else:
        urls_and_filenames.append((url, None))
        print(f"URL added: {url}")

Enter file URL (or type 'exit' to finish): https://f2l-bot828-40f1f76157f8.herokuapp.com/dl/68abb8a1f75526a5526e9e94
Do you want to modify the file name? (y/n): y
Enter new file name: photo-2025-08-25_01-13-04.jpg
URL and new filename added: https://f2l-bot828-40f1f76157f8.herokuapp.com/dl/68abb8a1f75526a5526e9e94 -> photo-2025-08-25_01-13-04.jpg
Enter file URL (or type 'exit' to finish): exit


In [21]:
print("\nAll collected URLs:")
for item in urls_and_filenames:
    if isinstance(item, tuple):
        print(f"URL: {item[0]}, New Filename: {item[1]}")
    else:
        print(f"URL: {item}")

print("\nStarting downloads...")
threads = []
for item in urls_and_filenames:
    if isinstance(item, tuple):
        url, new_filename = item
        t = threading.Thread(target=download, args=(url, new_filename))
    else:
        url = item
        t = threading.Thread(target=download, args=(url,))

    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("\nAll downloads finished.")


All collected URLs:
URL: https://f2l-bot828-40f1f76157f8.herokuapp.com/dl/68abb8a1f75526a5526e9e94, New Filename: photo-2025-08-25_01-13-04.jpg

Starting downloads...
photo-2025-08-25_01-13-04.jpg: 100%|██████████| 95.6k/95.6k [00:00<00:00, 314kB/s]

All downloads finished.
